In [ ]:
#@title Install required libraries
!pip install datasets
!pip install transformers
!pip install rouge_score
!pip install evaluate
!pip install sentencepiece
!pip install --no-cache-dir transformers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=6fc0f4e2aab4b4c8a67287d5e6de902a73558b61925f82ea706f814c474f9ce9
  Stored in directory: /root/.cache/pip

In [ ]:
from datasets import load_dataset

In [ ]:

data = load_dataset("csebuetnlp/xlsum", "burmese")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/4569 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/570 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/570 [00:00<?, ? examples/s]

In [ ]:
data.shape

{'train': (4569, 5), 'test': (570, 5), 'validation': (570, 5)}

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 4569
    })
    test: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 570
    })
    validation: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 570
    })
})

In [ ]:
# #@title Load data
# import pandas as pd
# from datasets import Dataset
# import datasets

# data = pd.read_csv('extractiveSummarization.csv')
# train_df = data[:576]
# test_df = data[576:]
# train_dataset = Dataset.from_dict(train_df)
# test_dataset = Dataset.from_dict(test_df)
# my_dataset_dict = datasets.DatasetDict({"train":train_dataset,"test":test_dataset})
# my_dataset_dict

DatasetDict({
    train: Dataset({
        features: ['Index', 'News', 'Category', 'category_id', 'Summary'],
        num_rows: 576
    })
    test: Dataset({
        features: ['Index', 'News', 'Category', 'category_id', 'Summary'],
        num_rows: 144
    })
})

In [ ]:
#@title Preprocessing the data
from transformers import AutoTokenizer

model_checkpoint = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=False)

max_input_length = 512
max_target_length = 30


def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["text"],
        max_length=max_input_length,
        truncation=True,
    )
    labels = tokenizer(
        examples["summary"], max_length=max_target_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = data.map(preprocess_function, batched=True)
tokenized_datasets

tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/4569 [00:00<?, ? examples/s]

Map:   0%|          | 0/570 [00:00<?, ? examples/s]

Map:   0%|          | 0/570 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 4569
    })
    test: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 570
    })
    validation: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 570
    })
})

Note:

In a lot of cases, you must be logged in with a Hugging Face account to interact with the Hub: download private repos, upload files, create PRs,… Create an account if you don’t already have one, and then sign in to get your User Access Token from your Settings page. The User Access Token is used to authenticate your identity to the Hub.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
#@title Fine-tuning mT5 with Keras
from transformers import TFAutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from transformers import create_optimizer
import tensorflow as tf
from transformers.keras_callbacks import PushToHubCallback

model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, from_pt=True)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

tokenized_datasets = tokenized_datasets.remove_columns(
    data["train"].column_names
)

features = [tokenized_datasets["train"][i] for i in range(2)]
data_collator(features)

tf_train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=5,
)
tf_eval_dataset = model.prepare_tf_dataset(
    tokenized_datasets["test"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=5,
)

# define training hyperparameters and compile
num_train_epochs = 3
num_train_steps = len(tf_train_dataset) * num_train_epochs
model_name = model_checkpoint.split("/")[-1]

optimizer, schedule = create_optimizer(
    init_lr=5.6e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)

model.compile(optimizer=optimizer)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

callback = PushToHubCallback(
    output_dir=f"/model/{model_name}-finetuned-bbcxlsumdata", tokenizer=tokenizer
)

model.fit(
    tf_train_dataset, validation_data=tf_eval_dataset, callbacks=[callback], epochs=5
)

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFMT5ForConditionalGeneration: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
- This IS expected if you are initializing TFMT5ForConditionalGeneration from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFMT5ForConditionalGeneration from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassificati

Epoch 1/5
913/913 [==============================] - 445s 460ms/step - loss: 6.3194 - val_loss: 2.8840
Epoch 2/5
913/913 [==============================] - 544s 596ms/step - loss: 4.3689 - val_loss: 2.6792
Epoch 3/5
913/913 [==============================] - 547s 599ms/step - loss: 4.0896 - val_loss: 2.6545
Epoch 4/5
913/913 [==============================] - 422s 462ms/step - loss: 4.0306 - val_loss: 2.6545
Epoch 5/5
913/913 [==============================] - 415s 454ms/step - loss: 4.0393 - val_loss: 2.6545


In [ ]:
#@title Using fine-tuned model
from transformers import pipeline

#pretrained_model = "csebuetnlp/mT5_multilingual_XLSum"     # pretrained model for myanmar language (mT5 checkpoint finetuned on the 45 languages of XL-Sum dataset)
pretrained_model = "brownniek/mt5-small-finetuned-bbcxlsumdata"    # lotaya pretrained model
summarizer = pipeline("summarization", model=pretrained_model)

text = "တီကျစ်ကျောက် မီးသွေးဓာတ်အားပေး စက်ရုံကို သက်တမ်းတိုး ထပ်မံလည်ပတ် ရန် ရှမ်းပြည်နယ်အစိုးရနှင့် ၀န် ကြီးဌာနက ကန့်ကွက်ရန်မရှိ ကြောင်း လျှပ်စစ်နှင့် စွမ်းအင်၀န် ကြီးဌာန ပြောရေးဆိုခွင့်နှင့် သတင်းပြန်ကြားရေး အဖွဲ့က ဇူလိုင် ၁၂ ရက်တွင် သတင်းထုတ်ပြန်သည်။ ရှမ်းပြည်နယ်ရှိ ကျောက်မီး သွေးသုံး ဓာတ်အားပေးစက်ရုံကို ၂၀၁၇ မေလ ၂၆ ရက်တွင် စမ်း သပ်လည်ပတ်ရာ တစ်နှစ်ပြည့် ၍ ထပ်မံသက်တမ်းတိုးရန် စီစဉ် နေပြီး လက်ရှိ၌ ပြည်ထောင်စု အစိုးရအဖွဲ့နှင့် သမ္မတရုံးသို့ တင်ပြထားကြောင်း လျှပ်စစ်ဓာတ် အားထုတ်လုပ်ရေးလုပ်ငန်း ဦးဆောင်ညွှန်ကြားရေးမှူး ဦးခင်မောင်၀င်းက ပြောသည်။ ‘‘အရင်တုန်းက ပြာတွေ ထွက်တယ်။ မီးခိုးနံ့တွေ ထွက် တယ်။ ဟိုတုန်းကစက်တွေ ဟောင်းစုတ်နေပါပြီ။ အခုစက်တွေပြင် ပြီးသွားပြီ။ ကမ္ဘာ့ဘဏ်သတ်မှတ် ချက်နှုန်း ပတ်၀န်းကျင်ထိန်းသိမ်း ရေး ၀န်ကြီးဌာနက သတ်မှတ်ထားတဲ့ ဂိုက်လိုင်းအတိုင်း ဒီအတွင်းထဲ မှာ ပြင်ပြီးသွားပြီ’’ဟု ၎င်းကဆက် လက်ပြောသည်။ အဆိုပါ စက်ရုံကို တရုတ် ကုမ္ပဏီက စနစ်တကျပြုပြင်ကာ အမျိုးသား ပတ်၀န်းကျင်ဆိုင်ရာ အရည်အသွေး (ထုတ်လွှတ်မှု) လမ်းညွှန်ချက်များနှင့် ကမ္ဘာ့ဘဏ်၏ သတ်မှတ်ချက် စံချိန်စံညွှန်းများအတိုင်း ဆောင်ရွက်ပြီး ဖြစ်ကြောင်း ဦးခင်မောင်၀င်းက ပြောသည်။ အဆိုပါ တီကျစ်ကျောက်မီး သွေးစက်ရုံကို ၂၀၁၅ မေလက တရုတ် Wuxi Hua Guang လျှပ် စစ်စွမ်းအင် ကုမ္ပဏီက တင်ဒါ အောင်မြင်ထားပြီး ၂၀၁၇ မေလ ၁၆ ရက်တွင် ဒေသခံများ၏ သဘောတူညီမှုဖြင့် တစ်နှစ်စမ်း သပ်လည်ပတ်ခွင့်ရခဲ့သည်။ ထိုစက်ရုံတွင် လျှပ်စစ်ဓာတ် အား စက်တပ်ဆင်အား မဂ္ဂါ၀ပ် ၁၂၀ ရှိပြီး လက်ရှိ၌ မဂ္ဂါ၀ပ် ၈၀ ထုတ်လုပ်နေသည်။ တရုတ် ကုမ္ပဏီ၏ လူမှုအကျိုးပြု အစီအ စဉ်ဖြင့် စက်ရုံပတ်လည်ရှိ ကျေး ရွာရှစ်ရွာတွင် လျှပ်စစ်မီးလင်းရေး ဆောင်ရွက်ခဲ့သည်ဟု ၀န်ကြီးဌာနက ဆိုထားသည်။ ထို့ကြောင့် ကျေးရွာများရှိ ဒေသခံအများစုက စက်ရုံကို ကန့်ကွက်မှုမရှိကြောင်း ဦးခင်မောင် ၀င်းက ဆိုသည်။"
summary = summarizer(text)

print(f"'Text: {text}'")
print(f"\n'Summary: {summary[0]['summary_text']}'")
print(f"\n'Summary: {summary}'")

config.json:   0%|          | 0.00/774 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/2.23G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers of TFMT5ForConditionalGeneration were initialized from the model checkpoint at brownniek/mt5-small-finetuned-bbcxlsumdata.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5ForConditionalGeneration for predictions without further training.


generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/833 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/416 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:838: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length.  recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'Text: တီကျစ်ကျောက် မီးသွေးဓာတ်အားပေး စက်ရုံကို သက်တမ်းတိုး ထပ်မံလည်ပတ် ရန် ရှမ်းပြည်နယ်အစိုးရနှင့် ၀န် ကြီးဌာနက ကန့်ကွက်ရန်မရှိ ကြောင်း လျှပ်စစ်နှင့် စွမ်းအင်၀န် ကြီးဌာန ပြောရေးဆိုခွင့်နှင့် သတင်းပြန်ကြားရေး အဖွဲ့က ဇူလိုင် ၁၂ ရက်တွင် သတင်းထုတ်ပြန်သည်။ ရှမ်းပြည်နယ်ရှိ ကျောက်မီး သွေးသုံး ဓာတ်အားပေးစက်ရုံကို ၂၀၁၇ မေလ ၂၆ ရက်တွင် စမ်း သပ်လည်ပတ်ရာ တစ်နှစ်ပြည့် ၍ ထပ်မံသက်တမ်းတိုးရန် စီစဉ် နေပြီး လက်ရှိ၌ ပြည်ထောင်စု အစိုးရအဖွဲ့နှင့် သမ္မတရုံးသို့ တင်ပြထားကြောင်း လျှပ်စစ်ဓာတ် အားထုတ်လုပ်ရေးလုပ်ငန်း ဦးဆောင်ညွှန်ကြားရေးမှူး ဦးခင်မောင်၀င်းက ပြောသည်။ ‘‘အရင်တုန်းက ပြာတွေ ထွက်တယ်။ မီးခိုးနံ့တွေ ထွက် တယ်။ ဟိုတုန်းကစက်တွေ ဟောင်းစုတ်နေပါပြီ။ အခုစက်တွေပြင် ပြီးသွားပြီ။ ကမ္ဘာ့ဘဏ်သတ်မှတ် ချက်နှုန်း ပတ်၀န်းကျင်ထိန်းသိမ်း ရေး ၀န်ကြီးဌာနက သတ်မှတ်ထားတဲ့ ဂိုက်လိုင်းအတိုင်း ဒီအတွင်းထဲ မှာ ပြင်ပြီးသွားပြီ’’ဟု ၎င်းကဆက် လက်ပြောသည်။ အဆိုပါ စက်ရုံကို တရုတ် ကုမ္ပဏီက စနစ်တကျပြုပြင်ကာ အမျိုးသား ပတ်၀န်းကျင်ဆိုင်ရာ အရည်အသွေး (ထုတ်လွှတ်မှု) လမ်းညွှန်ချက်များနှင့် ကမ္ဘာ့ဘဏ်၏ သတ်မှတ်ချက် စံချိန်စံညွှန်းများအတိုင်း ဆောင်ရွ

In [ ]:
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

article_text = """တီကျစ်ကျောက် မီးသွေးဓာတ်အားပေး စက်ရုံကို သက်တမ်းတိုး ထပ်မံလည်ပတ် ရန် ရှမ်းပြည်နယ်အစိုးရနှင့် ၀န် ကြီးဌာနက ကန့်ကွက်ရန်မရှိ ကြောင်း လျှပ်စစ်နှင့် စွမ်းအင်၀န် ကြီးဌာန ပြောရေးဆိုခွင့်နှင့် သတင်းပြန်ကြားရေး အဖွဲ့က ဇူလိုင် ၁၂ ရက်တွင် သတင်းထုတ်ပြန်သည်။ ရှမ်းပြည်နယ်ရှိ ကျောက်မီး သွေးသုံး ဓာတ်အားပေးစက်ရုံကို ၂၀၁၇ မေလ ၂၆ ရက်တွင် စမ်း သပ်လည်ပတ်ရာ တစ်နှစ်ပြည့် ၍ ထပ်မံသက်တမ်းတိုးရန် စီစဉ် နေပြီး လက်ရှိ၌ ပြည်ထောင်စု အစိုးရအဖွဲ့နှင့် သမ္မတရုံးသို့ တင်ပြထားကြောင်း လျှပ်စစ်ဓာတ် အားထုတ်လုပ်ရေးလုပ်ငန်း ဦးဆောင်ညွှန်ကြားရေးမှူး ဦးခင်မောင်၀င်းက ပြောသည်။ ‘‘အရင်တုန်းက ပြာတွေ ထွက်တယ်။ မီးခိုးနံ့တွေ ထွက် တယ်။ ဟိုတုန်းကစက်တွေ ဟောင်းစုတ်နေပါပြီ။ အခုစက်တွေပြင် ပြီးသွားပြီ။ ကမ္ဘာ့ဘဏ်သတ်မှတ် ချက်နှုန်း ပတ်၀န်းကျင်ထိန်းသိမ်း ရေး ၀န်ကြီးဌာနက သတ်မှတ်ထားတဲ့ ဂိုက်လိုင်းအတိုင်း ဒီအတွင်းထဲ မှာ ပြင်ပြီးသွားပြီ’’ဟု ၎င်းကဆက် လက်ပြောသည်။ အဆိုပါ စက်ရုံကို တရုတ် ကုမ္ပဏီက စနစ်တကျပြုပြင်ကာ အမျိုးသား ပတ်၀န်းကျင်ဆိုင်ရာ အရည်အသွေး (ထုတ်လွှတ်မှု) လမ်းညွှန်ချက်များနှင့် ကမ္ဘာ့ဘဏ်၏ သတ်မှတ်ချက် စံချိန်စံညွှန်းများအတိုင်း ဆောင်ရွက်ပြီး ဖြစ်ကြောင်း ဦးခင်မောင်၀င်းက ပြောသည်။ အဆိုပါ တီကျစ်ကျောက်မီး သွေးစက်ရုံကို ၂၀၁၅ မေလက တရုတ် Wuxi Hua Guang လျှပ် စစ်စွမ်းအင် ကုမ္ပဏီက တင်ဒါ အောင်မြင်ထားပြီး ၂၀၁၇ မေလ ၁၆ ရက်တွင် ဒေသခံများ၏ သဘောတူညီမှုဖြင့် တစ်နှစ်စမ်း သပ်လည်ပတ်ခွင့်ရခဲ့သည်။ ထိုစက်ရုံတွင် လျှပ်စစ်ဓာတ် အား စက်တပ်ဆင်အား မဂ္ဂါ၀ပ် ၁၂၀ ရှိပြီး လက်ရှိ၌ မဂ္ဂါ၀ပ် ၈၀ ထုတ်လုပ်နေသည်။ တရုတ် ကုမ္ပဏီ၏ လူမှုအကျိုးပြု အစီအ စဉ်ဖြင့် စက်ရုံပတ်လည်ရှိ ကျေး ရွာရှစ်ရွာတွင် လျှပ်စစ်မီးလင်းရေး ဆောင်ရွက်ခဲ့သည်ဟု ၀န်ကြီးဌာနက ဆိုထားသည်။ ထို့ကြောင့် ကျေးရွာများရှိ ဒေသခံအများစုက စက်ရုံကို ကန့်ကွက်မှုမရှိကြောင်း ဦးခင်မောင် ၀င်းက ဆိုသည်။"""

model_name = "brownniek/mt5-small-finetuned-bbcxlsumdata"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, from_pt=True)

input_ids = tokenizer(
    [WHITESPACE_HANDLER(article_text)],
    return_tensors="tf",
    padding="max_length",
    truncation=True,
    max_length=512
)["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    max_length=84,
    no_repeat_ngram_size=2,
    num_beams=4
)[0]

summary = tokenizer.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)

print(summary)


/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFMT5ForConditionalGeneration: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
- This IS expected if you are initializing TFMT5ForConditionalGeneration from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFMT5ForConditionalGeneration from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassificati

עם


In [ ]:
print(f"'summary: {summary}'")

'summary: [{'summary_text': ''}]'
